In [1]:
import glob
import re
import pandas as pd

In [2]:
##
## make dataframe of working areas
##

In [3]:
work_areas = glob.glob('work_areas/crab_201*')

In [4]:
def is_data(string):
    return 'data' in string
def get_era(string):
    eras = re.findall(r'(201[0-9])', string)
    if len(eras)==1: return eras[0]
    return 0

In [5]:
working_areas  = [{'area':w,'data': is_data(w), 'era':get_era(w)} for w in work_areas]
working_areas_df = pd.DataFrame(working_areas)

In [6]:
##
## test crab status
##

In [7]:
import os

In [8]:
def crab_read(string):
    stream = os.popen('crab status -d {}'.format(string))
    return stream.read()
test_string = 'work_areas/crab_2016_WJets_WJetsToQQ_HT_WJetsToQQ_HT-600ToInf_TuneCUETP8M1_13TeV-madgraphM_deepflavour_bff'

In [9]:
def get_status(string):
    results = re.findall(r'Jobs status: [ ]+([a-zA-Z]+)', string)
    if len(results)==1: return results[0]
    return 0
def job_counts(string):
    result = re.findall(r'\(([0-9]+)\/([0-9]+)\)', string)
    if len(result)==1: return result[0]
    return (0,0)
def jobs_finished(string):
    return job_counts(string)[0]
def jobs_total(string):
    return job_counts(string)[1]

In [10]:
def make_results(row):
    return crab_read(row.area)

In [11]:
#get results for each row
result_collumn = working_areas_df.apply(make_results, axis=1)

In [12]:
working_areas_df['results'] = result_collumn

In [13]:
working_areas_df['status'] = result_collumn.apply(get_status)
working_areas_df['jobs_finished'] = result_collumn.apply(jobs_finished)
working_areas_df['jobs_total'] = result_collumn.apply(jobs_total)

In [14]:
import subprocess

In [15]:
#print completed
delete = True
count = 0
for i,row in working_areas_df.iterrows():
    if row.status=='finished' and row.jobs_finished==row.jobs_total:
        count += 1
        print count, row.area
        if delete: 
            subprocess.check_output(['rm', '-rf', row.area])
            #subprocess.check_output(['mkdir', row.area])

1 work_areas/crab_2017_DYJets_DYJLL_M_50_nlo_DYJetsToLL_M-50_TuneCP5_1_deepflavour_bff
2 work_areas/crab_2017_DYJets_DYJLL_M_50_mad_v2_DYJetsToLL_M-50_TuneCP5_1_deepflavour_bff
3 work_areas/crab_2017_DYJets_DYJLL_M_100_200_DYJetsToLL_M-100to200_Tun_deepflavour_bff
4 work_areas/crab_2017_DYJets_DYJLL_M_200_400_DYJetsToLL_M-200to400_Tun_deepflavour_bff
5 work_areas/crab_2017_DYJets_DYJLL_M_400_500_DYJetsToLL_M-400to500_Tun_deepflavour_bff
6 work_areas/crab_2017_DYJets_DYJLL_M_500_700_DYJetsToLL_M-500to700_Tun_deepflavour_bff
7 work_areas/crab_2017_DYJets_DYJLL_M_800_1000_DYJetsToLL_M-800to1000_Tu_deepflavour_bff
8 work_areas/crab_2018_DYJets_DYJLL_M_50_nlo_DYJetsToLL_M-50_TuneCP5_1_deepflavour_bff
9 work_areas/crab_2018_DYJets_DYJLL_M_50_nlo_v2_DYJetsToLL_M-50_TuneCP5_1_deepflavour_bff
10 work_areas/crab_2018_DYJets_DYJLL_M_50_mad_DYJetsToLL_M-50_TuneCP5_1_deepflavour_bff
11 work_areas/crab_2018_DYJets_DYJLL_M_100_200_DYJetsToLL_M-100to200_Tun_deepflavour_bff
12 work_areas/crab_2018_DYJe

In [16]:
#print not complted
count = 0
for i,row in working_areas_df.iterrows():
    if not (row.status=='finished' and row.jobs_finished==row.jobs_total):
        count += 1
        print count, "\t", row.area, "\t",  row.status, "\t",  row.jobs_finished, "\t",  row.jobs_total 

1 	work_areas/crab_2017_DYJets_DYJLL_M_50_nlo_v2_DYJetsToLL_M-50_TuneCP5_1_deepflavour_bff 	finished 	121 	147
2 	work_areas/crab_2017_DYJets_DYJLL_M_50_mad_DYJetsToLL_M-50_TuneCP5_1_deepflavour_bff 	finished 	42 	43
3 	work_areas/crab_2017_DYJets_DYJLL_M_700_800_DYJetsToLL_M-700to800_Tun_deepflavour_bff 	failed 	14 	16
4 	work_areas/crab_2017_DYJets_DYJLL_M_50_nlo_v2_DYJetsToLL_M-50_TuneCP5_1_deepflavour_bff_eff 	finished 	145 	147
5 	work_areas/crab_2017_DYJets_DYJLL_M_50_mad_v2_DYJetsToLL_M-50_TuneCP5_1_deepflavour_bff_eff 	finished 	26 	34
6 	work_areas/crab_2018_DYJets_DYJLL_M_50_nlo_v2_DYJetsToLL_M-50_TuneCP5_1_deepflavour_bff_eff 	finished 	199 	201
7 	work_areas/crab_2018_DYJets_DYJLL_M_50_mad_DYJetsToLL_M-50_TuneCP5_1_deepflavour_bff_eff 	finished 	53 	57


In [17]:
#print failed
count = 0
for i,row in working_areas_df.iterrows():
    if (row.status=='failed'):
        count += 1
        print(count, row.area)

(1, 'work_areas/crab_2017_DYJets_DYJLL_M_700_800_DYJetsToLL_M-700to800_Tun_deepflavour_bff')


In [18]:
working_areas_df.to_csv('skim_status.csv')